

# TASI Lecture: Quantum teleportation in a quantum circuit

Author: J. Lykken

This notebook goes with Lecture One, June 23 2020, of 
"Quantum Information for Particle Theorists"
at the TASI summer school.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

[Cirq](https://urldefense.proofpoint.com/v2/url?u=https-3A__github.com_quantumlib_cirq&d=DwIGaQ&c=gRgGjJ3BkIsb5y6s49QqsA&r=Gl74vKKu9i-SToN0SgQC_w&m=ED3KJ2syYM_OQ64L0CIWSMP_ROvlcoHv83uHUp3_ryg&s=9kHGNyGghHB2ir0kC36e0CFYGNmbEIOnUVnvOrAausM&e=) is a framework for writing quantum algorithms for noisy intermediate scale quantum (NISQ) devices. 

---



### Connect to the Google Colab cloud GPU service

In [1]:
#### GOOGLE COLAB SPECIFIC ##########################
# You will need a free Google Colab account or a paid Colab Pro account
# Put this notebook in your Google Drive, then open it using Colab
#
# CODE SNIPPET TO ACCESS THE FILES IN YOUR GOOGLE DRIVE
# IT WILL ASK YOU FOR AUTHORIZATION VIA YOUR GOOGLE ACCOUNT

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
# !ls "/content/drive/My Drive"


Mounted at /content/drive


### Installing cirq

Executing the following cell will attempt to install cirq. On Google Colab it may then ask you to restart the runtime; do so.

In [2]:
!pip install cirq --quiet

     |████████████████████████████████| 1.6MB 2.7MB/s 


To verify that Cirq is installed in your environment, try to `import cirq` and print out a diagram of the 22-qubit Foxtail chip. It should produce a 2x11 grid of qubits.

In [3]:
import cirq
print(cirq.google.Foxtail)

(0, 0)───(0, 1)───(0, 2)───(0, 3)───(0, 4)───(0, 5)───(0, 6)───(0, 7)───(0, 8)───(0, 9)───(0, 10)
│        │        │        │        │        │        │        │        │        │        │
│        │        │        │        │        │        │        │        │        │        │
(1, 0)───(1, 1)───(1, 2)───(1, 3)───(1, 4)───(1, 5)───(1, 6)───(1, 7)───(1, 8)───(1, 9)───(1, 10)


### Import things we might need and define path to our Google Drive directory

In [4]:
# Define a googlepath to your googledrive to load/save files. 
# Modify the example below as needed
googlepath = "drive/My Drive/Colab Notebooks/TASI"
from cirq.circuits import InsertStrategy

import tensorflow as tf
import numpy as np
from numpy import around, log2, isreal, all, trace, conj, outer
from math import factorial, sqrt, pi
from cmath import exp
from numpy import matrix, eye, count_nonzero, around, sum, save
import scipy, sympy
from scipy import interpolate
from sympy import Symbol
import matplotlib.pyplot as plt
from matplotlib import gridspec
import gc, random, timeit
from timeit import default_timer as timer
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

### Experiment 1: Simplest single qubit circuit in Cirq
From Cirq tutorial and Jack Hidary's book Chapt 6

In [10]:
# Get a qubit and a circuit
qbit = cirq.LineQubit(0)
circuit = cirq.Circuit()

# Add an X gate: acts like the Pauli Matrix sigma_x
circuit.append(cirq.X(qbit))

# Run a simple simulation that extracts the wavefunction of this state
sim = cirq.Simulator()
result = sim.simulate(circuit)
printmd("\n**Bloch Sphere of the qubit in the final state:**")
state = cirq.bloch_vector_from_state_vector(result.final_state_vector,0)
print("x: ", around(state[0], 4), " y: ", around(state[1], 4),
      " z: ", around(state[2], 4))

# Add a measurement at the end of the circuit: 
circuit.append(cirq.measure(qbit, key="Final state"))

# Display the circuit:
printmd("\n**Cirq circuit:**")
print(circuit)

# Invoke the Cirq quantum simulator to execute the circuit:
simulator = cirq.Simulator()

# Simulate the circuit several times:
result = simulator.run(circuit, repetitions=10)

# Print the results:
printmd("\n**Results of 10 trials:**")
print(result)


**Bloch Sphere of the qubit in the final state:**

x:  0.0  y:  0.0  z:  -1.0



**Cirq circuit:**

0: ───X───M('Final state')───



**Results of 10 trials:**

Final state=1111111111


### Experiment 2: Circuit to create a Bell state in Cirq
From Jack Hidary's book Chapt 6

In [9]:
# Get two qubits and a circuit
qubit = [cirq.LineQubit(x) for x in range(2)]
circuit = cirq.Circuit()

# Add a Hadamard gate to qubit 0, then a CNOT gate from qubit 0 to qubit 1:
circuit.append([cirq.H(qubit[0]), 
             cirq.CNOT(qubit[0], qubit[1])])

# Run a simple simulation that extracts the actual final states
sim = cirq.Simulator()
result = sim.simulate(circuit)
printmd("\n**Bloch Sphere of the qubit 0 in the final state:**")
state = cirq.bloch_vector_from_state_vector(result.final_state_vector,0)
print("x: ", around(state[0], 4), " y: ", around(state[1], 4),
      " z: ", around(state[2], 4))
printmd("\n**Bloch Sphere of the qubit 1 in the final state:**")
state = cirq.bloch_vector_from_state_vector(result.final_state_vector,1)
print("x: ", around(state[0], 4), " y: ", around(state[1], 4),
      " z: ", around(state[2], 4))

# Add a measurement at the end of the circuit: 
circuit.append(cirq.measure(*qubit, key="Final state"))

# Display the circuit:
printmd("\n**Cirq circuit:**")
print(circuit)

# Invoke the Cirq quantum simulator to execute the circuit:
simulator = cirq.Simulator()

# Simulate the circuit several times:
result = simulator.run(circuit, repetitions=10)

# Print the results:
printmd("\n**Results:**")
print(result)


**Bloch Sphere of the qubit 0 in the final state:**

x:  0.0  y:  0.0  z:  0.0



**Bloch Sphere of the qubit 1 in the final state:**

x:  0.0  y:  0.0  z:  0.0



**Cirq circuit:**

0: ───H───@───M('Final state')───
          │   │
1: ───────X───M──────────────────



**Results:**

Final state=1000111111, 1000111111


### Experiment 3: Circuit to SWAP two qubit states

In [11]:
# Get two qubits and a circuit
qubit = [cirq.LineQubit(x) for x in range(2)]
circuit = cirq.Circuit()

# Add a Hadamard gate to make the initial state of qubit 0:
circuit.append([cirq.H(qubit[0])])

# Get a symbol
symbol = Symbol("t")
# Add a parameterized XPowGate to make the initial state of qubit 1:
circuit.append([cirq.XPowGate(exponent=symbol)(qubit[1])])

# Add three CNOT gates to make a SWAP gate:
circuit.append([cirq.CNOT(qubit[0], qubit[1]),
               cirq.CNOT(qubit[1], qubit[0]),
               cirq.CNOT(qubit[0], qubit[1])])

# Measure qubit 1 first, then measure qubit 0:
circuit.append(cirq.measure(qubit[1], key='q1'))
circuit.append(cirq.measure(qubit[0], key='q0'), strategy=InsertStrategy.NEW)

# Display the circuit:
printmd("\n**Cirq circuit:**")
print(circuit)

# Get a sweep over parameter values
sweep = cirq.Linspace(key=symbol.name, start=0.0, stop=1.0, length=3)

# Execute the circuit for all values in the sweep
sim = cirq.Simulator()
results = sim.run_sweep(circuit, sweep, repetitions=50)
printmd("\n**Results for t = 0:**")
print(results[0])
printmd("\n**Results for t = 1:**")
print(results[2])
printmd("\n**Results for t = 0.5:**")
print(results[1])




**Cirq circuit:**

0: ───H─────@───X───@─────────────M('q0')───
            │   │   │
1: ───X^t───X───@───X───M('q1')─────────────



**Results for t = 0:**

q0=00000000000000000000000000000000000000000000000000
q1=10110011100101111100100001001100101011100011110110



**Results for t = 1:**

q0=11111111111111111111111111111111111111111111111111
q1=00010001001010010011010001000011000010000001111001



**Results for t = 0.5:**

q0=00111110100111110101100000110001100001000011001010
q1=00010001110100101110111001111110001011100101000110


### Experiment 4: Quantum Teleportation
Quantum Teleportation is a process by which a quantum state can be transmitted
by sending only two classical bits of information. This is accomplished by
pre-sharing an entangled state between the sender (Alice) and the receiver
(Bob). This entangled state allows the receiver (Bob) of the two classical
bits of information to possess a qubit with the same state as the one held by
the sender (Alice).

In the following example output, qubit 0 (the Message) is set to a random state
by applying X and Y gates. By sending two classical bits of information after
a Bell State Measurement of qubit 0 (the Message) and qubit 1 (Alice's entangled qubit), the
final state of qubit 2 (Bob's entangled qubit) will be identical to the
original random state of qubit 0 (the Message). This is only possible given
that an entangled state is pre-shared between Alice and Bob.

Example adapted from the Cirq tutorials

In [12]:
# Define three qubits: msg = qubit[0], qalice = qubit[1], qbob = qubit[2]
qubit=[0]*(3)
qubit[0] = cirq.NamedQubit('msg')
qubit[1] = cirq.NamedQubit('qalice')
qubit[2] = cirq.NamedQubit('qbob')

circuit = cirq.Circuit()
# Create a Bell state entangled pair to be shared between Alice and Bob.
circuit.append([cirq.H(qubit[1]), cirq.CNOT(qubit[1], qubit[2])])

# Creates a random state for the Message.
ranX = random.random()
ranY = random.random()
circuit.append([cirq.X(qubit[0])**ranX, cirq.Y(qubit[0])**ranY])

# Unitary operator rotating the two-qubit basis of the Message and Alice's entangled qubit;
# rotates the Bell state basis to the computational basis:
circuit.append([cirq.CNOT(qubit[0], qubit[1]), cirq.H(qubit[0])])
# Combining now with a measurment in the computational basis,
# we effectively have projected this two-qubit state onto one of the four states of
# the Bell state basis:
circuit.append(cirq.measure(qubit[0], qubit[1]))

# Use the two classical bits from the Bell measurement to recover the
# original quantum Message on Bob's entangled qubit.
circuit.append([cirq.CNOT(qubit[1], qubit[2]), cirq.CZ(qubit[0], qubit[2])])

printmd("\n**Cirq circuit:**")
print(circuit)

sim = cirq.Simulator()

# Run a simple simulation that applies the random X and Y gates that
# create our message.
q0 = cirq.LineQubit(0)
message = sim.simulate(cirq.Circuit([cirq.X(q0)**ranX, cirq.Y(q0)**ranY]))

printmd("\n**Bloch Sphere of the Message qubit in the initial state:**")
expected = cirq.bloch_vector_from_state_vector(message.final_state_vector,0)
print("x: ", around(expected[0], 4), " y: ", around(expected[1], 4),
      " z: ", around(expected[2], 4))

# Records the final state of the simulation.
final_results = sim.simulate(circuit)

printmd("\n**Bloch Sphere of Bob's qubit in the final state:**")
teleported = cirq.bloch_vector_from_state_vector(
    final_results.final_state_vector, 2)
print("x: ", around(teleported[0], 4), " y: ",
    around(teleported[1], 4), " z: ", around(teleported[2], 4))

printmd("\n**Bloch Sphere of the Message qubit in the final state:**")
message_final = cirq.bloch_vector_from_state_vector(
    final_results.final_state_vector, 0)
print("x: ", around(message_final[0], 4), " y: ",
    around(message_final[1], 4), " z: ", around(message_final[2], 4))


**Cirq circuit:**

msg: ──────X^0.565───Y^0.176───@───H───M───────@───
                               │       │       │
qalice: ───H─────────@─────────X───────M───@───┼───
                     │                     │   │
qbob: ───────────────X─────────────────────X───@───



**Bloch Sphere of the Message qubit in the initial state:**

x:  -0.1066  y:  -0.9793  z:  -0.1721



**Bloch Sphere of Bob's qubit in the final state:**

x:  -0.1066  y:  -0.9793  z:  -0.1721



**Bloch Sphere of the Message qubit in the final state:**

x:  0.0  y:  0.0  z:  -1.0


### Experiment 5: Quantum Teleportation without measurement


In [14]:
# Define three qubits: msg = qubit[0], qalice = qubit[1], qbob = qubit[2]
qubit=[0]*(3)
qubit[0] = cirq.NamedQubit('msg')
qubit[1] = cirq.NamedQubit('qalice')
qubit[2] = cirq.NamedQubit('qbob')

circuit = cirq.Circuit()
# Create a Bell state entangled pair to be shared between Alice and Bob.
circuit.append([cirq.H(qubit[1]), cirq.CNOT(qubit[1], qubit[2])])

# Creates a random state for the Message.
ranX = random.random()
ranY = random.random()
circuit.append([cirq.X(qubit[0])**ranX, cirq.Y(qubit[0])**ranY])

# Unitary operator rotating the two-qubit basis of the Message and Alice's entangled qubit;
# rotates the Bell state basis to the computational basis:
circuit.append([cirq.CNOT(qubit[0], qubit[1]), cirq.H(qubit[0])])
# But this time skip the measurement
# circuit.append(cirq.measure(qubit[0], qubit[1]))

# Use the same operations as before to recover the
# original quantum Message on Bob's entangled qubit.
circuit.append([cirq.CNOT(qubit[1], qubit[2]), cirq.CZ(qubit[0], qubit[2])])

printmd("**Cirq circuit:**")
print(circuit)

sim = cirq.Simulator()

# Run a simple simulation that applies the random X and Y gates that
# create our message.
q0 = cirq.LineQubit(0)
message = sim.simulate(cirq.Circuit([cirq.X(q0)**ranX, cirq.Y(q0)**ranY]))

printmd("\n**Bloch Sphere of the Message qubit in the initial state:**")
expected = cirq.bloch_vector_from_state_vector(message.final_state_vector,0)
print("x: ", around(expected[0], 4), " y: ", around(expected[1], 4),
      " z: ", around(expected[2], 4))

# Records the final state of the simulation.
final_results = sim.simulate(circuit)

printmd("\n**Bloch Sphere of Bob's qubit in the final state:**")
bob_final = cirq.bloch_vector_from_state_vector(
    final_results.final_state_vector, 2)
print("x: ", around(bob_final[0], 4), " y: ",
    around(bob_final[1], 4), " z: ", around(bob_final[2], 4))

printmd("\n**Bloch Sphere of the Message qubit in the final state:**")
message_final = cirq.bloch_vector_from_state_vector(
    final_results.final_state_vector, 0)
print("x: ", around(message_final[0], 4), " y: ",
    around(message_final[1], 4), " z: ", around(message_final[2], 4))

printmd("\n**Bloch Sphere of the Alice qubit in the final state:**")
alice_final = cirq.bloch_vector_from_state_vector(
    final_results.final_state_vector, 1)
print("x: ", around(alice_final[0], 4), " y: ",
    around(alice_final[1], 4), " z: ", around(alice_final[2], 4))

**Cirq circuit:**

msg: ──────X^0.531───Y^0.277───@───H───@───
                               │       │
qalice: ───H─────────@─────────X───@───┼───
                     │             │   │
qbob: ───────────────X─────────────X───@───



**Bloch Sphere of the Message qubit in the initial state:**

x:  -0.0732  y:  -0.9954  z:  -0.0618



**Bloch Sphere of Bob's qubit in the final state:**

x:  -0.0732  y:  -0.9954  z:  -0.0618



**Bloch Sphere of the Message qubit in the final state:**

x:  1.0  y:  0.0  z:  0.0



**Bloch Sphere of the Alice qubit in the final state:**

x:  1.0  y:  0.0  z:  0.0
